In [ ]:
import heapq as hq
import networkx as nx
import matplotlib.pyplot as plt
# für 
from random import shuffle 

class Event:
    def __init__(self, time, action, bm, passenger):
        self.time = time
        self.action = action
        self.bm = bm
        self.p = passenger
    def __lt__(self, other):
        return (self.time, 
                self.bm,
                self.p) < (other.time,
                           other.bm,
                           other.p)
    def __le__(self, other):
        return (self.time,
                self.bm,
                self.p) <= (other.time, 
                            other.bm,
                            other.p)

class passagier:
    def __init__(self, Station):
        self.station = Station
        #self.umstieg = {'Umstieg': False, 'Station': Station}
        #self.umstieg = False
        self.time = 0
        self.bm = 'hallo'
    def __lt__(self, other):
        return (self.time,
                self.bm,
                self.station) < (other.time,
                                 other.bm,
                                 other.station) #and bool_lt(self.umstieg,other.umstieg)
    def __le__(self, other):
        return (self.time,
                self.bm, 
                self.station) <= (other.time, 
                                  other.bm, 
                                  other.station) #and bool_le(self.umstieg,other.umstieg)
        
def bool_le(self, other):
    if self == other:
        return True
    elif (self,other) == (True,False):
        return True
    else:
        return False

def bool_lt(self, other):
    if self == other:
        return False
    elif (self,other) == (True,False):
        return True
    else:
        return False
        
def shuffle0(passengers):
    for i in range(len(passengers)):
        x = [j for j in range(len(passengers[i]))]
        shuffle(x)
        passengers[i] = [passengers[i][k] for k in x]

def calc_pic(grah, ListeColor, ListeSize):
    #nx.draw(graph, pos, with_labels = True, node_color = ListeColor, node_size = ListeSize)
    nx.draw_kamada_kawai(graph, with_labels = True, node_color = ListeColor, node_size = ListeSize)
    
#default='#ecf2e3'
default='#cad3b8'

In [ ]:
# Lösung für Passger als Classe
def solve(eventq, passengers, bms, graph):
    
    v = [len([idx for idx, passenger in enumerate(passengers[n]) if passenger.station == n]) == 0 for n in range(len(passengers))]
    if not (v == [True]*len(passengers)):
        ErrorMessage = 'Bei Passengers ist man Knoten {} Passagiere die in gleiche Sationen'.format([idx for idx, boolean in enumerate(v) if boolean == False])
        raise Exception(ErrorMessage)
    
    if not (len(graph.node)) == (len(passengers)):
        ErrorMessage = 'Die Graphiken werden Fehler haft berrechnet.\n'
        if len(graph.node) < len(passengers):
            ErrorMessage += 'Es sind zu viele Einträge in Passengers ({}) als Knoten in Graph ({})'.format(len(passengers),len(graph.node)) + '\n'
            ErrorMessage += 'Löschen/Fügen Sie ({}) Einträge/Knoten in Passengers/Graph'.format(abs(len(passengers) - len(graph.node)))
        else:
            ErrorMessage += 'Es sind zu wenige Einträge in Passengers ({}) als Knoten in Graph ({})'.format(len(passengers),len(graph.node)) + '\n'
            ErrorMessage += 'Fügen/Löschen Sie ({}) Einträge/Knoten in Passengers/Graph'.format(abs(len(passengers) - len(graph.node)))
        raise Exception(ErrorMessage)
    
    if not [len(bms[i]['itinerary']) == len(bms[i]['travel_times']) for i in range(len(bms))] == [True]*len(bms):
        raise Exception('In der Liste BMS ist itinerary und travel_times nicht gleich groß')
    
    #TODO Umsteigen
    
    
    ####
    # Visualisierung
    ListeNodes = list(graph.node)
    ListeColor = [default]*len(ListeNodes)
    ####
    
    zykel = 0
    while eventq:
        print('Durch läuft die While-Schleine zum {} mal'.format(zykel))
        # Abbruch Bedingung
        if ([len(passengers[i]) == 0 for i in range(len(passengers))] == [True]*len(passengers)) and ([len(bms[i]['n_passengers']) == 0 for i in range(len(bms))] == [True]*len(bms)):
            return print('Ende')
        
        event = hq.heappop(eventq)
        
        Bahnen = [idx for idx, bahn in enumerate(bms) if bahn['name'] == event.bm]
        if not Bahnen:
            raise Exception('Fehler')
        if len(Bahnen)>1:
            raise Exception('Fehler')
        bm = bms[Bahnen[0]]

        itinerary_len = len(bm['itinerary'])
        current_node = bm['itinerary'][bm['stop'] % itinerary_len]
        cn_number = [idx for idx, station in enumerate(bm['itinerary']) if station == current_node][0] #Current Node Number
        
        next_node = bm['itinerary'][(bm['stop']+1) % itinerary_len]
        nn_number = [idx for idx, station in enumerate(bm['itinerary']) if station == next_node][0]
        print('Fahrzeug {} ist am Knoten {}, der nächste Halt {}, die aktuelle Zeit: {}'.format(bm['name'],current_node, next_node, event.time))
        print('und {} Stopp zum {} mal'.format(bm['name'],bm['stop']))
        
        ####
        # Visualisierung
        ListeSize = [len(i)*10 for i in passengers]
        ListeColor[current_node] = bm['color']
        #calc_pic(graph, ListeColor, ListeSize)
        #plt.savefig("img/{}.png".format(event.time))
        #plt.show()
        ListeColor[current_node] = default
        ####
        
        if event.action == 'load':
            #print('Program ist im Zustand: {}'.format(event.action))
            free = bm['capacity'] - len(bm['n_passengers'])
            print('Fahrzeug {} load'.format(bm['name']))
            print('Fahrzeug {} hat {} Plätze frei'.format(bm['name'],free))
            while len(bm['n_passengers']) < bm['capacity']:
                
                if not current_node < len(passengers):
                    raise Exception('Die Länge von passengers {} ist <= current_node {}'.format(len(passengers),current_node))
                
                waiting = [idx for idx, passenger in enumerate(passengers[current_node]) if passenger.station in bm['itinerary']]
                #print('Warten: {}'.format(waiting))
                if not waiting:
                    break
                passenger = passengers[current_node].pop(waiting[0])
                passenger.bm = bm['name']
                
                exit_node = passenger.station
                en_nubmer = [idx for idx, station in enumerate(bm['itinerary']) if station == exit_node][0]
                
                ListExit = []    
                for i in range((en_nubmer-cn_number)% itinerary_len):
                    ListExit += [ bm['travel_times'][(cn_number + i) % itinerary_len] ]
                ExitTime = event.time - 2
                for i in ListExit:
                    ExitTime += (i)
                #print('Exit Time: {}, und ListExit {}'.format(ExitTime,ListExit))
                
                passenger.time = ExitTime
                #hq.heappush(bm['n_passengers'], passenger)
                #bm['n_passengers'].sort()
                hq.heappush(bm['n_passengers'], ExitTime)
                bm['n_passengers'].sort()
                event_neu = Event(ExitTime,'exit', bm['name'], passenger)
                if not event_neu.time == passenger.time:
                    raise Exception('help')
                if not event_neu.bm == passenger.bm:
                    raise Exception('nope')
                hq.heappush(eventq, event_neu)
                
            
            hq.heappush(eventq, Event(event.time + bm['travel_times'][nn_number], 'load', bm['name'], passagier(_)))
            bm['stop'] += 1
            print('Es Warten noch {} an {}'.format(len(passengers[current_node]),current_node))
        
        # Generate events for exiting passengers
        
        elif event.action == 'exit':
            """
            i = 0
            while i==0:
                GetOut = [idx for idx, passenger in enumerate(bm['n_passengers']) if (passenger.time == event.time) and (passenger.station == current_node)]
                if not GetOut:
                    print('Something is Wrong, len "GetOut" {}'.format(len(GetOut)))
                    passenger = hq.heappop(bm['n_passengers'])
                    #raise Exception('Unknown event action')
                    break
                passenger = bm['n_passengers'].pop(GetOut[0])
                i+=1
            """
            passenger = event.p
            #print('alt gleich neu {}'.format(p == passenger))
            print(bm['n_passengers'])
            ExitTime = hq.heappop(bm['n_passengers'])
            #print('PassagierStation:{} PassagierTime:{}, Umstieg {} steit aus, EventTime {},{}'.format(passenger.station, passenger.time, passenger.umstieg,event.time,ExitTime))
            print('PassagierStation:{} PassagierTime:{} steit aus, EventTime {},{}'.format(passenger.station, passenger.time,event.time,ExitTime))
            print('Ist EvenTime == PassagierTime: {}'.format(passenger.time == event.time))
            if not passenger.time == event.time:
                print(passenger)
                bahn = passenger.bm
                print(bahn,bm['name'])
                Bahnen = [idx for idx, bahn in enumerate(bms) if bahn['name'] == passenger.bm]
                if not Bahnen:
                    raise Exception('Fehler')
                if len(Bahnen)>1:
                    raise Exception('Fehler')
                bm0 = bms[Bahnen[0]]
                print(bm0['n_passengers'])
                
            print('Passagier am richtigen Ort: {}'.format(passenger.station == current_node))
            if [passenger.station == current_node] == [False]:
                passengers[current_node] += [passenger]
                shuffle0(passengers)
                print('Passagier mit Ziel: {}, wurde falsch berechnet.'.format(passenger.station))
            print('Es sind {} Passagiere in der Bahn'.format(len(bm['n_passengers'])))
            
            
        else:
            raise Exception('Unknown event action')
        eventq.sort()
        zykel += 1

In [ ]:
graph = nx.Graph()

graph.add_node('Hbf', pos=(10,10))  #0
graph.add_node('INF', pos=(10,30))  #1
graph.add_node('Phil', pos=(30,30))  #2
graph.add_node('Uni', pos=(40,20))  #3
graph.add_node('Bism', pos=(30,10))  #4
graph.add_node('Rohr', pos=(30,0))  #5
graph.add_node('Hand', pos=(15,40))  #6
graph.add_node('Zieg', pos=(50,30))  #7

ListeNodes = list(graph.node)
#Rundfahrt 
graph.add_edge(ListeNodes[0],ListeNodes[1])
graph.add_edge(ListeNodes[1],ListeNodes[0])
graph.add_edge(ListeNodes[1],ListeNodes[2])
graph.add_edge(ListeNodes[2],ListeNodes[1])
graph.add_edge(ListeNodes[2],ListeNodes[3])
graph.add_edge(ListeNodes[3],ListeNodes[2])
graph.add_edge(ListeNodes[3],ListeNodes[4])
graph.add_edge(ListeNodes[4],ListeNodes[3])
graph.add_edge(ListeNodes[4],ListeNodes[0])
graph.add_edge(ListeNodes[0],ListeNodes[4])

#Zubringer
graph.add_edge(ListeNodes[4],ListeNodes[5])
graph.add_edge(ListeNodes[5],ListeNodes[4])
graph.add_edge(ListeNodes[1],ListeNodes[6])
graph.add_edge(ListeNodes[6],ListeNodes[1])
graph.add_edge(ListeNodes[3],ListeNodes[7])
graph.add_edge(ListeNodes[7],ListeNodes[3])


pos=nx.get_node_attributes(graph,'pos')
#nx.draw(graph,pos,with_label=True)

In [ ]:
bms = [{'color': '#f0f000', 'n_passengers': [], 'capacity': 30, 'stop': 0, 'itinerary': [4, 3, 2, 1, 0], 'name': 'zykel0', 'travel_times': [5, 5, 6, 6, 7]},
    {'color': '#f000f0', 'n_passengers': [], 'capacity': 30, 'stop': 0, 'itinerary': [0, 1, 2, 3, 4], 'name': 'zykel1', 'travel_times': [7, 7, 6, 6, 5]},
    {'color': '#ff0000', 'n_passengers': [], 'capacity': 30, 'stop': 0, 'itinerary': [4,5], 'name': 'zubinger0', 'travel_times': [5, 5]},
    {'color': '#00ff00', 'n_passengers': [], 'capacity': 30, 'stop': 0, 'itinerary': [1,6], 'name': 'zubinger1', 'travel_times': [5, 6]},
    {'color': '#0000ff', 'n_passengers': [], 'capacity': 30, 'stop': 0, 'itinerary': [3,7], 'name': 'zubinger2', 'travel_times': [6, 5]}
       ]#

passengers = [#
    [passagier(1)]*30 +[passagier(2)]*140 +[passagier(3)]*10, #0
    [passagier(0)]*30 +[passagier(2)]*30 +[passagier(3)]*10, #1
    [passagier(0)]*50 +[passagier(1)]*20 +[passagier(3)]*10, #2
    [passagier(0)]*60 +[passagier(2)]*20 +[passagier(1)]*10, #3
    [passagier(0)]*10 +[passagier(2)]*10 +[passagier(3)]*10, #4
    [passagier(4)]*20, #5
    [passagier(1)]*20, #6
    [passagier(3)]*20  #7
               ]

# shuffle passengers
shuffle0(passengers)


In [ ]:
a = passagier(0)
event0 = Event(0,'load','zykel0',_)
event1 = Event(0,'load','zykel1',_)
event2 = Event(0,'load','zubinger0',_)
event3 = Event(0,'load','zubinger1',_)
event4 = Event(0,'load','zubinger2',_)
eventq = []
hq.heappush(eventq, event0)
hq.heappush(eventq, event1)
hq.heappush(eventq, event2)
hq.heappush(eventq, event3)
hq.heappush(eventq, event4)


solve(eventq, passengers, bms, graph)